# Библиотеки и установка

In [1]:
from sentence_transformers import SentenceTransformer, util
import torch

In [2]:
#pip install nltk

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
#import pymorphy2
from nltk.stem.snowball import RussianStemmer
from collections import Counter
from sklearn.metrics import confusion_matrix
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [4]:
# Загрузка необходимых ресурсов
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

# Запишем путь, скачаем

In [5]:
from google.colab import drive #для колаба
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
zip_path = '/content/drive/MyDrive/lenta-ru-news.csv.zip'

In [7]:
import zipfile
import os

extract_dir = '/content/data'  # папка, куда будет извлечён архив
os.makedirs(extract_dir, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

In [8]:
#считаем файлы
df = pd.read_csv(
    '/content/data/lenta-ru-news.csv',

)

<ipython-input-8-b51ab22b0819>:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(


In [9]:
df=df.copy()
df.shape

(800975, 6)

In [10]:
#слишком шумные топики (наблюдений менее 10)
remove_topics = ['Сочи', 'ЧМ-2014', 'МедНовости', 'Оружие']
df = df[~df['topic'].isin(remove_topics)]

In [11]:
df=df.dropna(axis=0)#избавимся от н/ашек
df.shape

(712647, 6)

# Пытаемся сделать описания классов (для топ 5)


In [12]:
#пытаемся добыть описания классов. Сначала ищем самые часто встречающиеся слова.
#затем вручную на их основе вводим какое-то описание и затем на основе этого ищем топ-5.
#


In [13]:
russian_stopwords = stopwords.words('russian')
#по этой функции ищем топ 15 слов, которые характеризуют каждый из топиков
#(латентное размещение Дирихле)
def get_topic_words(texts, n_words=15):
    vectorizer = CountVectorizer(stop_words=russian_stopwords, max_df=0.95, min_df=1)
    X = vectorizer.fit_transform(texts)
    lda = LatentDirichletAllocation(n_components=1, random_state=42)
    lda.fit(X)
    terms = vectorizer.get_feature_names_out()
    topic = lda.components_[0]
    top_indices = topic.argsort()[-n_words:][::-1]
    top_words = [terms[i] for i in top_indices]
    return top_words
#нужно было раньше (в 3 топиках было слишком мало примеров меньше 10)
def fallback_top_words(texts, n_words=10):
    words = " ".join(texts).lower().split()
    words = [w for w in words if w not in russian_stopwords and w.isalpha()]
    common = Counter(words).most_common(n_words)
    return [w for w, _ in common]



In [14]:
for label in df['topic'].unique():
    texts = df[df['topic'] == label]['text'].dropna().tolist()
    n = len(texts)
    if n >= 10:
        top_words = get_topic_words(texts[:min(500, n)])
        print(f"{label} (LDA, {n} текстов) — {', '.join(top_words)}")
    else:
        top_words = fallback_top_words(texts)
        print(f"{label} (fallback, {n} текстов) — {', '.join(top_words)}")

Библиотека (LDA, 65 текстов) — года, 1914, время, сентября, войны, журнал, это, немцы, война, солдат, петроград, солдаты, русский, несколько, сентябрь
Россия (LDA, 155004 текстов) — россии, также, новости, риа, сегодня, словам, это, рф, заявил, сообщил, москвы, года, пресс, время, москве
Мир (LDA, 136620 текстов) — сша, сообщает, года, это, также, время, однако, человек, итар, тасс, долларов, году, заявил, которые, тысяч
Экономика (LDA, 76423 текстов) — россии, года, долларов, процентов, сообщает, рублей, также, компании, миллионов, агентство, время, это, году, 2000, заявил
Интернет и СМИ (LDA, 44421 текстов) — интернет, сообщает, 2000, также, года, сша, это, microsoft, компании, com, однако, время, которые, будут, ru
Спорт (LDA, 57894 текстов) — матче, чемпионата, матч, однако, мира, матча, россии, место, лишь, сборной, команды, минуте, турнира, второй, чемпионате
Культура (LDA, 53530 текстов) — фильм, фильме, фильма, году, это, также, роль, режиссер, кино, года, который, режиссера, ф

In [15]:
category_descriptions = {
    'Библиотека':'Материалы об исторических событиях и жизни в прошлом, особенно в контексте военных лет, Петрограда, солдат и хронологии начала XX века.',
    'Россия':'Новости и события, связанные с внутренней политикой, обществом, заявлениями официальных лиц и жизнью в российских городах, включая Москву.',
    'Мир':'Международная повестка: события за рубежом, политика США, мировая экономика, конфликты и международные отношения.',
    'Экономика':'Вопросы, касающиеся российского и глобального бизнеса, рубля, доллара, цен, инфляции,ВВП а также деятельности компаний и рыночных трендов.',
    'Интернет и СМИ':'Темы, связанные с интернетом, СМИ, цифровыми технологиями и крупными IT-компаниями (например, Microsoft), а также новостными порталами.',
    'Спорт':'События из мира спорта: матчи, турниры, чемпионаты, сборные команды и результаты соревнований.',
    'Культура':'Кино, театр, музыка, роли актеров и режиссёров. Анонсы и обсуждения культурных событий и персоналий.',
    'Наука и техника':'Новости науки и технологий, исследования, космос (МКС, NASA), разработки и мнения учёных.',
    'Бизнес':'Экономическая активность компаний, инвестиции, отчетность, валютные потоки, изменения в бизнес-среде.',
    'Из жизни':'Лайфстайл, интересные события, происшествия, человеческие истории и нестандартные новости.',
    'Бывший СССР':'Политические и социальные события в странах бывшего Советского Союза: Украина, Молдова, Киргизия и другие',
    'Дом':'Недвижимость, строительство, стоимость жилья, квадратные метры, компании-застройщики и рынок жилья.',
    'Путешествия':'Темы туризма, поездок, путешествий внутри России и за границу, статистика и потребительское поведение туристов.',
    'Силовые структуры':'Работа МВД, судов, следственных органов, дела против чиновников, правонарушения и силовая политика.',
    'Ценности':'Мода, бренды, коллекции одежды, часы, товары класса люкс и потребительское поведение в этой сфере.',
    'Легпром':'Новости лёгкой промышленности, производства текстиля и одежды, статистика Минпромторга и экспортно-импортная динамика.',
    'Культпросвет':'Деятельность Минкультуры, общественные и культурно-просветительские инициативы, высказывания чиновников.',
    'Крым':'События в Крыму и Севастополе, деятельность местных властей, политика интеграции и региональное управление на полуострове.',
    '69-я параллель':'Жизнь и события в Мурманской области и других северных регионах, арктическая повестка, бюджет и развитие инфраструктуры.'
}

In [16]:
#выгружаем предобученную модель
model_1 = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
index_to_class = list(category_descriptions.keys())
descriptions = list(category_descriptions.values())
desc_embeddings = model_1.encode(descriptions, convert_to_tensor=True, normalize_embeddings=True)

# Функция предсказания топ-5 классов
#внутри функции считается косинусное сходство между векторами текста и векторами описаний
def predict_dense(text, top_k=5):
    query_embedding = model_1.encode(text, convert_to_tensor=True, normalize_embeddings=True)
    scores = util.cos_sim(query_embedding, desc_embeddings)[0]
    top_results = torch.topk(scores, k=top_k)
    return [(index_to_class[i], category_descriptions[index_to_class[i]]) for i in top_results.indices]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.89k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [17]:
text = "Яркие полосы, которые десятилетиями считались возможным признаком текущей воды на Марсе, скорее всего имеют совсем иное происхождение. Исследование, опубликованное в Nature Communications, показало: эти структуры образуются в результате процессов, связанных с ветром и пылью, а не с влагой. "
print(predict_dense(text))

[('Наука и техника', 'Новости науки и технологий, исследования, космос (МКС, NASA), разработки и мнения учёных.'), ('69-я параллель', 'Жизнь и события в Мурманской области и других северных регионах, арктическая повестка, бюджет и развитие инфраструктуры.'), ('Дом', 'Недвижимость, строительство, стоимость жилья, квадратные метры, компании-застройщики и рынок жилья.'), ('Экономика', 'Вопросы, касающиеся российского и глобального бизнеса, рубля, доллара, цен, инфляции,ВВП а также деятельности компаний и рыночных трендов.'), ('Интернет и СМИ', 'Темы, связанные с интернетом, СМИ, цифровыми технологиями и крупными IT-компаниями (например, Microsoft), а также новостными порталами.')]


# LLM

In [18]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch
#загружаем модель и ее токенизатор
model_name = "t-tech/T-lite-it-1.0"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)

#также создадим пайплай генерации ради упрощения использования модели
generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

tokenizer_config.json:   0%|          | 0.00/4.90k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/712 [00:00<?, ?B/s]

...

model-00004-of-00004.safetensors:   0%|          | 0.00/1.09G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.87G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

Device set to use cuda:0


# Интеграция с retrieval-моделью

In [19]:
#в этой функции мы пытаемся объединить 2 модели (2 этапа)
#сначала retrieval-модель ищет топ 5, затем с помощью ллм из 5 ыбираем финальный класс
#также пишем промпт для ллм, чтобы она выбрала итоговую (включает текст, список категорий)
def predict_with_llm(text, top_k=5):
    top_candidates = predict_dense(text, top_k=top_k)

    prompt = f"Определи, к какой из следующих категорий относится текст:\n\n"
    prompt += f"Текст: \"{text}\"\n\nКатегории:\n"
    for i, (label, desc) in enumerate(top_candidates, 1):
        prompt += f"{i}. {label} — {desc}\n"
    prompt += "\nВыбери номер подходящей категории и укажи только её название.\nОтвет:"

    output = generator(
        prompt,
        max_new_tokens=20,
        temperature=0.3,
        top_p=0.9,
        do_sample=False,
        return_full_text=False
    )

    result = output[0]["generated_text"].strip()

    for label, desc in top_candidates:
        if label.lower() in result.lower():
            return label, desc

    return top_candidates[0]

In [20]:
example = "Президент России Владимир Путин прибыл в Волгоград ранним утром 1 января 2014 года, сообщает РИА Новости. Путин провел совещание по борьбе с терроризмом, возложил цветы на месте, где 30 декабря был взорван троллейбус, а затем навестил в больнице пострадавших при волгоградских терактах. Поездка не была анонсирована заранее.В антитеррористическом совещании приняли участие главы ФСБ и МВД Александр Бортников и Владимир Колокольцев, начальник ГУ МВД РФ по Волгоградской области Александр Кравченко, начальник УФСБ по области Сергей Блинов, министр здравоохранения России Вероника Скворцова и губернатор региона Сергей Боженов."
print(predict_with_llm(example))

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.3` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:653: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `70` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(


('Россия', 'Новости и события, связанные с внутренней политикой, обществом, заявлениями официальных лиц и жизнью в российских городах, включая Москву.')


# Попробуем глянуть на предсказания для текста

In [21]:
#мы слишком оптимистоично взяли такой большой датасет, поэтому проверить полностью, насколько оно лучше бейслайна не получится
#тк оно будет предсказывать очень долго,вот часть примеров

In [24]:
sample_df = df.sample(n=6).reset_index(drop=True)

In [25]:

results = []

for _, row in sample_df.iterrows():
    text = row['text']
    true_label = row['topic']

    predicted_label, predicted_desc = predict_with_llm(text, top_k=5)

    results.append({
        'text': text,
        'true_label': true_label,
        'predicted_label': predicted_label,
        'predicted_description': predicted_desc,
        'match': str(true_label).lower() == str(predicted_label).lower()
    })

# 3. Преобразуем в DataFrame
results_df = pd.DataFrame(results)

# 4. Выводим таблицу
print(results_df[['true_label', 'predicted_label', 'match', 'predicted_description']])

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.3` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:653: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `70` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a 

          true_label  predicted_label  match  \
0           Из жизни  Наука и техника  False   
1     69-я параллель   69-я параллель   True   
2  Силовые структуры           Россия  False   
3             Россия           Россия   True   
4          Экономика        Экономика   True   
5        Бывший СССР              Мир  False   

                               predicted_description  
0  Новости науки и технологий, исследования, косм...  
1  Жизнь и события в Мурманской области и других ...  
2  Новости и события, связанные с внутренней поли...  
3  Новости и события, связанные с внутренней поли...  
4  Вопросы, касающиеся российского и глобального ...  
5  Международная повестка: события за рубежом, по...  


In [26]:
with pd.option_context('display.max_colwidth', None):
    display(results_df[['text', 'true_label', 'predicted_label', 'match', 'predicted_description']])

,text,true_label,predicted_label,match,predicted_description
0,"Новозеландская компания Martin Aircraft намерена в 2012 году начать продажи индивидуальных летательных аппаратов Martin Jetpack. Ubergizmo пишет, что ""летающий ранец"" обойдется покупателю в 100 тысяч долларов. Martin Jetpack, изобретенный основателем новозеландской компании Гленом Мартином, впервые был представлен в 2008 году. Для того, чтобы Martin Jetpack не попал под ограничения для гражданской авиации, его скорость полета была ограничена примерно 100 километрами в час, а человека аппарат может поднять на высоту в 2,5 километра. Максимальная длительность полета Martin Jetpack составляет 30 минут, а работает аппарат на бензине. Экспериментальный образец представлял собой устройство массой 115 килограммов. Небольшие габариты освободят покупателя Martin Jetpack от необходимости обзаводиться специальной лицензией. В Martin Aircraft, работавшей над проектом более 25 лет, рассчитывают, что покупателями летательных аппаратов станут не только частные лица, но и спасательные службы. Кроме того, в Martin Aircraft считают, что аппаратом могут заинтересоваться военные.",Из жизни,Наука и техника,False,"Новости науки и технологий, исследования, космос (МКС, NASA), разработки и мнения учёных."
1,"Площадь мемориального комплекса «Долина Славы» в 40 километрах от Мурманска, планируют увеличить в 2018 году для захоронения непогребенных воинов, погибших при защите Заполярья в годы Великой Отечественной войны. Об этом ТАСС сообщил председатель координационного совета поисковых отрядов Мурманской области Константин Добровольский. «В Долине Славы изначально было создано шесть секторов, площадь каждого примерно по 450 квадратных метров, — рассказал Добровольский. — Еще три года назад мы говорили о том, что места для новых захоронений может не хватить, а сегодня его практически не осталось». По его словам, поисковые работы в Заполярье не прекращались, начиная с 1959 года. Поисковики работают в местах, где проходили ожесточенные бои с фашистами. За это время захоронены останки уже более 20 тысяч воинов, из них около 7 тысяч — в «Долине Славы», еще около 15 тысяч солдат и офицеров не найдены. За время поискового сезона в этом году были обнаружены останки 87 советских воинов, погибших в период с 1941 по 1944 годы. Трое из них были опознаны по личным вещам. Это летчик, младший лейтенант Овчинников Петр Иванович, красноармеец Петухов Иван Кириллович и красноармеец Безумов Федор Дмитриевич. Мемориал «Долина Славы» расположен на правом берегу реки Западная Лица. Ранее это место называлось Долиной Смерти — здесь шли ожесточенные бои на главном участке фронта в Мурманской операции 1941 года. Немецкие войска в годы войны так и не смогли захватить город с его незамерзающим портом, железную дорогу, а также главную базу Северного флота в Полярном.",69-я параллель,69-я параллель,True,"Жизнь и события в Мурманской области и других северных регионах, арктическая повестка, бюджет и развитие инфраструктуры."
2,"Министр экономического развития России Алексей Улюкаев, обвиняемый во взяточничестве, во вторник, 15 ноября, в Басманном суде Москвы рассказал о своих болезнях, сообщает РИА Новости. «У меня имеются заболевания: атеросклероз, гипертония и заболевания, в целом связанные с сосудистой системой», — сказал Улюкаев. В этой связи он попросил суд не помещать его под домашний арест. В свою очередь его адвокат Тимофей Гриднев предложил ограничиться подпиской о невыезде, поскольку министру по состоянию здоровья необходимы ежедневные прогулки минимум по два часа. Кроме того, считает защитник, отсутствие Улюкаева на работе дестабилизирует деятельность министерства. Между тем прокуратура, как сообщалось, поддержала ходатайство следствия об избрании в отношении Улюкаева меры пресечения в виде домашнего ареста. О задержании министра сообщил Следственный комитет в ночь на 15 ноября. По версии следствия, Улюкаев незаконно потребовал от представителя компании «Роснефть» в качестве взятки 2 миллиона долларов за з

In [ ]:
#в рамках простой классификации работает не всегда хорошо (возможно еще причина в том, что топики часто пересекаются).
#в целом модель улавливает контекст и даже если предсказанная категория не сопоставляется True, то контекст предсказанной категории в тексте тоже присутсвует

In [27]:
#адаптивная классификация в рамках этого датасета необходима, так как часто в разные периоды времени одни топики сильно предобладают над другими
#также в датасете был дисбаланс классов, и адаптивная классификация позволяет качественнее выявлять более редко встречающиеся